<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Self_Attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn

class SelfAttention(nn.Module):
    def __init__(self, input_dim, attention_dim):
        super(SelfAttention, self).__init__()
        self.query = nn.Linear(input_dim, attention_dim)
        self.key = nn.Linear(input_dim, attention_dim)
        self.value = nn.Linear(input_dim, attention_dim)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, x):
        Q = self.query(x)
        K = self.key(x)
        V = self.value(x)
        attention_scores = torch.bmm(Q, K.transpose(1, 2))
        attention_weights = self.softmax(attention_scores)
        attention_output = torch.bmm(attention_weights, V)
        return attention_output

# Example usage
input_data = torch.randn(10, 5, 64)  # Batch size of 10, sequence length of 5, input dimension 64
self_attention = SelfAttention(input_dim=64, attention_dim=32)
output = self_attention(input_data)
print("Self-attention output shape:", output.shape)  # Expected shape: [10, 5, 32]